En aquesta pràctica treballareu amb un model de detecció d’objectes basat en PyTorch i la xarxa neuronal VGG16. L'objectiu es modificar el model per adaptar-lo a un problema de detecció d'objectes específic utilitzant un conjunt de dades simples.

![Exemple](08_Detecció/imgs/img.png)

Emprarem un *dataset* de detecció d'objectes que conté imatges d'estrelles. Aquest conjunt de dades és senzill i ideal per a practicar tècniques de detecció d'objectes. El podeu trobar a Kaggle al següent [enllaç](https://www.kaggle.com/datasets/kishanj/simple-object-detection). Per carregar aquest tipus de dataset haurem d'implementar una classe personalitzada que hereti de `torch.utils.data.Dataset`.

In [ ]:
from torch.utils.data import Dataset

class EstrellesDataset(Dataset):
    def __init__(self, transforms):
        super().__init__()
        self.transform = transforms
        # TODO

    def __len__(self):
        # TODO

    def __getitem__(self, idx):
        # TODO

        return img, label, bbox

## Preparació del model

Començarem carregant el model VGG16 preentrenat i adaptant-lo per a la detecció d'objectes. Afegirem capes addicionals per predir les caixes delimitadores (bounding boxes) i les classes dels objectes.

In [ ]:
from torch import nn
from torchvision import models, transforms

vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
backbone = vgg16.features

class VGG16ObjectDetector(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.avgpool = # TODO

        # Capes fully-connected de la VGG original
        self.flatten = nn.Flatten()
        self.fc =

        # Cap de classificació
        self.class_head =

        # Cap de regressió de bounding box
        self.bbox_head =

    def forward(self, x):
        #TODO

        return class_logits, bbox_preds

VGG16ObjectDetector(backbone=backbone)